In [1]:
import pandas as pd
import numpy as np

In [2]:
## Setting up the working directory

import os
#os.chdir('/mnt/d/Dropbox (Partners HealthCare)/impression_bert/recist')

### Reading the original RECIST annotation datafile

1. Using pd.read_csv to read the respective datafiles
2. Using error_bad_lines = False to skip occasional bad lines


In [3]:
recist_labels = pd.read_csv('timc_data/timepointreport.csv', error_bad_lines = False, low_memory=False)
pt_reg = pd.read_csv('../../profile_2022/oncdrs_data/REQ_KK71_105304_PT_INFO_STATUS_REGISTRATION.csv', low_memory=False)

/tmp/ipykernel_5470/2794614779.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  recist_labels = pd.read_csv('timc_data/timepointreport.csv', error_bad_lines = False, low_memory=False)
Skipping line 1165: expected 14 fields, saw 16
Skipping line 1347: expected 14 fields, saw 15
Skipping line 1479: expected 14 fields, saw 15
Skipping line 4840: expected 14 fields, saw 15
Skipping line 6228: expected 14 fields, saw 15
Skipping line 6239: expected 14 fields, saw 15
Skipping line 6249: expected 14 fields, saw 15
Skipping line 10127: expected 14 fields, saw 16
Skipping line 10902: expected 14 fields, saw 15
Skipping line 11134: expected 14 fields, saw 16
Skipping line 11789: expected 14 fields, saw 15
Skipping line 11898: expected 14 fields, saw 15
Skipping line 12813: expected 14 fields, saw 15
Skipping line 15781: expected 14 fields, saw 15
Skipping line 15883: expected 14 fields, saw 15
Skipp

In [4]:
pt_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51006 entries, 0 to 51005
Data columns (total 50 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   PATIENT_ID                     51006 non-null  int64  
 1   LAST_NM                        51006 non-null  object 
 2   FIRST_NM                       51005 non-null  object 
 3   MIDDLE_NM                      21010 non-null  object 
 4   TITLE_NM                       109 non-null    object 
 5   BIRTH_DT                       51006 non-null  object 
 6   D_BIRTH_DT                     51006 non-null  float64
 7   AGE90_BIRTH_DT                 51006 non-null  object 
 8   HYBRID_DEATH_IND               51006 non-null  object 
 9   HYBRID_DEATH_DT                20866 non-null  object 
 10  D_HYBRID_DEATH_DT              20866 non-null  float64
 11  AGE90_HYBRID_DEATH_DT          51006 non-null  object 
 12  HYBRID_DEATH_SOURCE            20866 non-null 

### Formatting the registration datafile 

1. Creating a new column using the same name (MRN) as in timepoint datafile
2. Converting the MRN numbers to numeric/integers as they are in numeric/integer format in timepoint (non-numeric will be converted into NaN)
3. Replacing NaNs with zeros
4. Converting MRN values into integers (originally as float)

In [5]:
pt_reg['MRN'] = pt_reg['BWH_MRN']
pt_reg['MRN'] = pd.to_numeric(pt_reg['MRN']).fillna(0) ## need to make all MRNs numeric
                                                                   ## with pd.to_numeric, MRNs which are not numeric will be converted to NaN                                                                    
pt_reg['MRN'] = pt_reg['MRN'].dropna()
pt_reg['MRN'] = pt_reg['MRN'].astype(int)

### Merging the two datafiles (timepoint and PT_INFO_STATUS_REGISTRATION)

1. The purpose of this merge is to get DFCI MRN linked to the RECIST annotations, which are indexed with a BWH MRN by default. 
2. Using pd.merge to inner merge timepoint, and registration datafiles by the common column, 'MRN'


In [ ]:
recist_reg = pd.merge(recist_labels, 
                          pt_reg,
                   on='MRN')



In [ ]:
recist_reg = recist_reg[['Protocol', 
                 'DFCI_MRN', 
                 'VisitDate', 
                 'OverallResponse']]

recist_reg = recist_reg.rename(columns = {'VisitDate': 'date', 
                                          'DFCI_MRN': 'dfci_mrn', 
                                          'OverallResponse': 'overall_response',
                                         'Protocol': 'protocol'})

recist_reg['date'] = pd.to_datetime(pd.Series(recist_reg.date), 
                           errors = 'coerce', 
                           format = "%Y-%m-%d")


def to_string(date):
    if date:
        string = date.strftime("%Y-%m-%d")
    else:
        string = ""

    return string

recist_reg['date'] = recist_reg['date'].map(to_string)



### Parsing/Reading single JSON file

1. Get the file handler for a file on disk
2. Load the content of the file by using the file handler
3. Convert the JSON file into pd dataframe
4. Export the dataframe as csv (not required but can be helpful)

### Parsing/Reading multiple JSON files

1. Create a list of JSON file paths
2. Initiate an empty list 'dfs' to be appended later
3. Run a for loop -
    For each JSON file in the list of JSON files
        open each JSON file
        load each JSON file
        convert each JSON file into a pd Df
        Append the list 'dfs' with each JSON file
4. Concatenate (combined) multiple JSON files into a dataframe
5. Export dataframe as csv

In [9]:
import json

fns = [
    '../../profile_2022/text_data/RequestID-105304_Imag_File1.json',
    '../../profile_2022/text_data/RequestID-105304_Imag_File2.json',
    '../../profile_2022/text_data/RequestID-105304_Imag_File3.json',
    '../../profile_2022/text_data/RequestID-105304_Imag_File4.json',
]

dfs = []
for fn in fns:
    print('* parsing %s' % fn)
    # get the file handler for a file on disk
    fh = open(fn, encoding='utf8')
    
    # load the content of the file by using that file handler
    j = json.load(fh)

    # to save/convert a list of objects/dictionaries
    # we will convert it first to a Pandas DataFrame first
    dft = pd.DataFrame(j['response']['docs'])
    
    # combine the dft to a list
    dfs.append(dft)

# combine all df in the dfs into one df
df = pd.concat(dfs)
print('* combined df')

* parsing ../../profile_2022/text_data/RequestID-105304_Imag_File1.json
* parsing ../../profile_2022/text_data/RequestID-105304_Imag_File2.json
* parsing ../../profile_2022/text_data/RequestID-105304_Imag_File3.json
* parsing ../../profile_2022/text_data/RequestID-105304_Imag_File4.json
* combined df


### Next steps

1. The next step is to link the text data from the parsed JSON files with the RECIST labels from the timepoint datafile
2. This must be done at the multiple timepoints because each patient (DFCI_MRN) can have multiple scans

In [10]:
json_df = df.rename(columns = {'EVENT_DATE': 'date',
                         'DFCI_MRN': 'dfci_mrn'})

json_df['dfci_mrn'] = pd.to_numeric(json_df['dfci_mrn']).fillna(0)
json_df['dfci_mrn'] = json_df['dfci_mrn'].astype(int)


json_df['date'] = pd.to_datetime(pd.Series(json_df.date), ##Converting dates into datetime format
                           errors = 'coerce', 
                           format = "%Y-%m-%d")

def to_string(date): ##Defining a function to convert datetime back to string if date or else leave it empty
    if date:
        string = date.strftime("%Y-%m-%d")
    else:
        string = ""

    return string

json_df['date'] = json_df['date'].map(to_string) ##Mapping function to date

In [ ]:
json_df.info()

In [ ]:
json_df = json_df[["date", ##Keeping only relevant columns from json_df
         "dfci_mrn",
        'RPT_TEXT',
        "NARRATIVE_TEXT",
        "IMPRESSION_TEXT",
        "ADDENDUM_TEXT"]]

json_df.head()

## Merge JSON (imaging reports) to RECIST annotations

In [15]:
##Inner merge on date and MRN
labeled_data = pd.merge(recist_reg, 
                    json_df,
                    on = ['date', 'dfci_mrn'],
                   how = 'inner')



In [16]:
## Renaming column of interest
labeled_data = labeled_data.rename(columns = {'NARRATIVE_TEXT': 'narrative_text', 'RPT_TEXT':'report_text', 'IMPRESSION_TEXT':'impression_text'})

In [18]:
## Getting counts of RECIST labels
print('The total number of unique DFCI MRNs are:', len(labeled_data['dfci_mrn'].unique())), labeled_data['overall_response'].value_counts()

The total number of unique DFCI MRNs are: 5163


(None,
 SD        30867
 BL        25185
 PR        23517
 PD        15290
 CR         4459
 NN         1205
 SC         1186
 NE          707
 UN          639
 PDu         226
 NP          184
 POS          29
 CRu          17
 FS           10
 -64.9%        6
 -95%          6
 -80.1%        5
 -100%         5
 NEG           2
 0%            1
 Name: overall_response, dtype: int64)

In [19]:
# drop obs's (trial-treatment combos) that have any non-standard annotations
no_list = ['-64.9%', 
                                                                   '-95%', 
                                                                   '-80.1%', 
                                                                   '-100%', 
                                                                   '0%',
                                                                  'NN',
                                                                  'SC',
                                                                  'NE',
                                                                  'UN',
                                                                  'PDu',
                                                                  'NP',
                                                                  'POS',
                                                                  'CRu',
                                                                  'FS',
                                                                  'NEG']

labeled_data['drop_this_obs'] = labeled_data.groupby(['protocol','dfci_mrn']).overall_response.transform(lambda x: (x.isin(no_list).any()))




In [20]:
labeled_data.drop_this_obs.value_counts()

False    90149
True     13605
Name: drop_this_obs, dtype: int64

In [21]:
## Removing irrelevant labels

labeled_data = labeled_data[~labeled_data.drop_this_obs]
# labeled_data = labeled_data[labeled_data['overall_response'].isin(['-64.9%', 
#                                                                    '-95%', 
#                                                                    '-80.1%', 
#                                                                    '-100%', 
#                                                                    '0%',
#                                                                   'NN',
#                                                                   'SC',
#                                                                   'NE',
#                                                                   'UN',
#                                                                   'PDu',
#                                                                   'NP',
#                                                                   'POS',
#                                                                   'CRu',
#                                                                   'FS',
#                                                                   'NEG']) == False]

print("Number of entries:", labeled_data.shape[0])


labeled_data['overall_response'] = labeled_data['overall_response'].dropna()
labeled_data = labeled_data[labeled_data['overall_response'].notna()]

print("Number of entries:", labeled_data.shape[0])

Number of entries: 90149
Number of entries: 89943


In [22]:
# merge to predefined patient-level train/val/test splits
splits = pd.read_csv('/mnt/d/Dropbox (Partners HealthCare)/profile_9-2022/derived_data/split_9-2022.csv')[['dfci_mrn','split']]

In [ ]:
labeled_data=pd.merge(labeled_data, splits, on='dfci_mrn', how='inner').drop(['ADDENDUM_TEXT'], axis=1)


In [27]:
# create an identifier corresponding to unique combos of MRN and protocol numbers
#patient trial combination
#one to many relationship (patients to obs)
#one to one (obs to pat)
copy = labeled_data.copy()
labeled_data['obs'] = labeled_data.groupby(['dfci_mrn','protocol']).ngroup()
labeled_data = labeled_data.sort_values(by=['obs','date'])


In [28]:
labeled_data.obs.isnull().value_counts()

False    89943
Name: obs, dtype: int64

In [29]:
labeled_data.split.value_counts()

train         72529
test           9044
validation     8370
Name: split, dtype: int64

In [30]:
## Getting the unique recist labels
labeled_data['overall_response'].unique()

array(['BL', 'SD', 'PD', 'PR', 'CR'], dtype=object)

In [31]:
## Converting categorical variable values (recist labels) to numerical codes
labeled_data['overall_response_factor'] = labeled_data['overall_response'].replace([
                        'BL', 'PR', 'CR', 'SD', 'PD'],
                        [0, 1, 2, 3, 4])

In [32]:
# Check variable info
labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89943 entries, 19994 to 57168
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   protocol                 89943 non-null  object
 1   dfci_mrn                 89943 non-null  int64 
 2   date                     89943 non-null  object
 3   overall_response         89943 non-null  object
 4   report_text              89943 non-null  object
 5   narrative_text           89940 non-null  object
 6   impression_text          58413 non-null  object
 7   drop_this_obs            89943 non-null  bool  
 8   split                    89943 non-null  object
 9   obs                      89943 non-null  int64 
 10  overall_response_factor  89943 non-null  int64 
dtypes: bool(1), int64(3), object(7)
memory usage: 7.6+ MB


In [36]:
labeled_data = labeled_data.groupby(['dfci_mrn','date','report_text']).first().reset_index()

In [37]:
labeled_data.to_csv('./timc_recist_labeled_dataset_id.csv')